### Map Graphs

Allows us to present data on top of graphs, or to visualise differences between countries on a world map.

For this, we need to use another python visualisation library called Folium.

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [3]:
import numpy as np
import pandas as pd

In [4]:
world_map=folium.Map()

world_map

One of the cool things about *Folium* is that it produces a map that is interactive, and allows you to zoom in and out. You can customize this also by defining the initial zoom level of your map, plus the center of your map, by using latitude and longitude values. The higher the zoom level used, the more the map will be zoomed to the center that you have defined.

In [5]:
#center the map around Canada
world_map=folium.Map(location=[56.130,-106.35],zoom_start=4)

world_map

Different styles of maps can also be created, let's try some out:

- Stamen Toner Maps - high contrast black and white:

In [6]:
world_map=folium.Map(location=[56.130,-106.35],zoom_start=4,tiles='Stamen Toner')

world_map

- Stamen Terrain Maps - featuring hill shading and natural vegetation:

In [8]:
world_map=folium.Map(location=[56.130,-106.35],zoom_start=4,tiles='Stamen Terrain')

world_map

It is also possible to make annotations on our maps.

We are going to try not to use a speadsheet to annoate a map.

Let's start with importing a police department incidents data report:

In [12]:
df_incidents=pd.read_csv('~/Documents/Cambridge/Python-Lessons/Police_Department_Incidents.csv')

#sanity check that it worked
print('Dataset downloaded and read into a pandas dataframe')

Dataset downloaded and read into a pandas dataframe


In [13]:
df_incidents.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


In [14]:
#how many entries are there in our dataset?
df_incidents.shape

(150500, 13)

In [15]:
#select the first 100 rows to work with
df_incidents=df_incidents.iloc[0:100,:]

In [17]:
df_incidents.shape

(100, 13)

Let's try to visualise the crimes that took place in the city of San Francisco.

In [18]:
#create the map
sanfran_map=folium.Map(location=[37.77,-122.42],zoom_start=12)

sanfran_map

In [20]:
#create a feature group containing the incidents
incidents=folium.map.FeatureGroup()

#loop through the 100 crimes
for lat, lng in zip(df_incidents.Y,df_incidents.X):
    incidents.add_child(
    folium.features.CircleMarker(
    [lat,lng],
    radius=5,
    color='yellow',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6))
    
#add incidents to the map
sanfran_map.add_child(incidents)    

Next, we will try to add some pop-up text to each marker, so that when we click the marker, we can see what category of crime that was:

In [28]:
#start with the same that was done before
#create a feature group containing the incidents
incidents=folium.map.FeatureGroup()

#loop through the 100 crimes
for lat, lng in zip(df_incidents.Y,df_incidents.X):
    incidents.add_child(
    folium.features.CircleMarker(
    [lat,lng],
    radius=5,
    color='yellow',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6))
    
#add pop-up text
latitudes=list(df_incidents.Y)
longitudes=list(df_incidents.X)
labels=list(df_incidents.Category)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng],popup=label).add_to(sanfran_map)
    
#like before, now add incidents to map
sanfran_map.add_child(incidents)

Similarly, this can also be done but without the location markers.

In [27]:
sanfran_map=folium.Map(location=[37.77,-122.42],zoom_start=12)

for lat, lng,label in zip(df_incidents.Y,df_incidents.X,df_incidents.Category):
    folium.features.CircleMarker(
    [lat,lng],
    radius=5,
    color='yellow',
    fill=True,
        popup=label,
    fill_color='blue',
    fill_opacity=0.6).add_to(sanfran_map)
    
sanfran_map

We can also group markers into different clusters.

In [31]:
from folium import plugins
sanfran_map=folium.Map(location=[37.77,-122.42],zoom_start=12)

#create a mark cluster objet
incidents=plugins.MarkerCluster().add_to(sanfran_map)

#add data point to the mark clusters
for lat, lng, label, in zip(df_incidents.Y,df_incidents.X,df_incidents.Category):
    folium.Marker(
    location=[lat,lng],
    icon=None,
    popup=label,).add_to(incidents)
    
sanfran_map

Once you click on the clusters, the map will zoom in and show you smaller and smaller clusters, until you are at the level of the location markers, which you can then click on to see the pop up text.